In [1]:
import colorsys
import os
import re
import pickle
from timeit import default_timer as timer

import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
from keras.utils import multi_gpu_model, plot_model
# from tensorflow.python.keras.utils.vis_utils import plot_model

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [7]:
print("Configuration...")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.66
sess = tf.Session(config=config)
K.set_session(sess)

Configuration...


In [2]:
num_anchors = 3
num_classes = 1
base_model = tiny_yolo_body(Input(shape=(None, None, 3)), 3, 1)
base_model.load_weights('logs/000/trained_weights_final_part2.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
with open("weights_part1.pickle",mode="rb") as f:
    data = pickle.load(f)

In [4]:
type(base_model.layers[1].get_weights()[0])

numpy.ndarray

In [5]:
for conv_layer in data["conv"].keys():
    layer_id = data["conv"][conv_layer]["layer_id"]
    target_layer = base_model.layers[layer_id]
    print(layer_id,conv_layer)
    print(len(data["conv"][conv_layer]["weight"]))
    
    overwrite = []
    for i in range(len(data["conv"][conv_layer]["weight"])):
        overwrite.append((data["conv"][conv_layer]["weight"][i] + base_model.layers[layer_id].get_weights()[i])/2)
        
    base_model.layers[layer_id].set_weights(overwrite)

1 conv2d_1
1
5 conv2d_2
1
9 conv2d_3
1
13 conv2d_4
1
17 conv2d_5
1
21 conv2d_6
1
25 conv2d_7
1
28 conv2d_8
1
31 conv2d_11
1
36 conv2d_9
1
37 conv2d_12
1
42 conv2d_10
2
43 conv2d_13
2


In [6]:
base_model.save_weights('logs/000/' + 'merged_model_1and2.h5')

In [7]:
data["conv"]["conv2d_10"]["weight"][0] + base_model.layers[42].get_weights()[0]

array([[[[ 0.01488509,  0.09312313, -0.12623265, ..., -0.04523799,
           0.03232977,  0.03840299],
         [ 0.13872363, -0.20751075, -0.02205791, ...,  0.10523109,
           0.07886406, -0.10988729],
         [-0.05915035, -0.16401802,  0.05193938, ...,  0.09470253,
           0.06922428, -0.11107668],
         ...,
         [-0.19359607,  0.14992426, -0.19649108, ..., -0.08281652,
           0.05204211, -0.0902181 ],
         [-0.10532525,  0.01468646, -0.19235545, ...,  0.11316811,
          -0.06182029,  0.09179034],
         [-0.14878207,  0.02384858,  0.03380951, ..., -0.09103279,
           0.08436612,  0.1287271 ]]]], dtype=float32)

In [8]:
base_model.layers[42].name

'conv2d_10'

In [19]:
len(data["conv"]["conv2d_10"]["weight"][0].shape)

AttributeError: 'list' object has no attribute 'shape'

In [12]:
len(base_model.layers[42].get_weights()[0].shape)

4

In [24]:
len(data["conv"]["conv2d_10"]["weight"])

2